#Use aggregate functions

** Data Source **
* English Wikipedia pageviews by second
* Size on Disk: ~255 MB
* Type: Parquet files
* More Info: <a href="https://datahub.io/en/dataset/english-wikipedia-pageviews-by-second" target="_blank">https&#58;//datahub.io/en/dataset/english-wikipedia-pageviews-by-second</a>

**Technical Accomplishments:**
* Introduce the various aggregate functions.

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Getting Started

Run the following cell to configure our "classroom."

In [0]:
%run "./Includes/Classroom-Setup"

Initialized classroom variables & functions...

In [0]:
# Mount "/mnt/training" again using "%run "./Includes/Dataset-Mounts-New"" if it is failed in "./Includes/Classroom-Setup"
try:
    files = dbutils.fs.ls("/mnt/training")
except:
    dbutils.fs.unmount('/mnt/training/')


/mnt/training/ has been unmounted.


Mounted datasets to /mnt/training from wasbs://training@dbtrainwesteurope.blob.core.windows.net/

In [0]:
%run "./Includes/Dataset-Mounts-New"

Created user-specific database

Using the database vishal_abnave_borregaard_com_db .

All done!

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) The Data Source

This data uses the **Pageviews By Seconds** data set.

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# I've already gone through the exercise to determine
# how many partitions I want and in this case it is...
partitions = 8

# Make sure wide operations don't repartition to 200
spark.conf.set("spark.sql.shuffle.partitions", str(partitions))

In [0]:
(source, sasEntity, sasToken) = getAzureDataSource()
spark.conf.set(sasEntity, sasToken)

source = '/mnt/training'
# The directory containing our parquet files.
parquetFile = source + "/wikipedia/pageviews/pageviews_by_second.parquet/"

In [0]:
# Create our initial DataFrame. We can let it infer the 
# schema because the cost for parquet files is really low.
initialDF = (spark.read
  .option("inferSchema", "true") # The default, but not costly w/Parquet
  .parquet(parquetFile)          # Read the data in
  .repartition(partitions)       # From 7 >>> 8 partitions
  .cache()                       # Cache the expensive operation
)
# materialize the cache
initialDF.count()

# rename the timestamp column and cast to a timestamp data type
pageviewsDF = (initialDF
  .withColumnRenamed("timestamp", "capturedAt")
  .withColumn("capturedAt", unix_timestamp( col("capturedAt"), "yyyy-MM-dd'T'HH:mm:ss").cast("timestamp") )
)

# cache the transformations on our new DataFrame by marking the DataFrame as cached and then materialize the result
pageviewsDF.cache().count()

7200000

In [0]:
display(pageviewsDF)

capturedAt,site,requests
2015-03-30T09:39:32.000+0000,desktop,2343
2015-04-01T08:26:41.000+0000,mobile,811
2015-03-30T11:18:38.000+0000,mobile,914
2015-04-02T16:22:24.000+0000,mobile,1415
2015-03-28T23:42:23.000+0000,desktop,2067
2015-03-26T16:30:21.000+0000,desktop,3121
2015-03-28T02:25:49.000+0000,desktop,1743
2015-03-27T07:56:26.000+0000,desktop,1929
2015-03-26T12:51:16.000+0000,desktop,2713
2015-03-26T18:43:56.000+0000,mobile,1313


##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) groupBy()

Aggregating data is one of the more common tasks when working with big data.
* How many customers are over 65?
* What is the ratio of men to women?
* Group all emails by their sender.

The function `groupBy()` is one tool that we can use for this purpose.

If you look at the API docs, `groupBy(..)` is described like this:
> Groups the Dataset using the specified columns, so that we can run aggregation on them.

This function is a **wide** transformation - it will produce a shuffle and conclude a stage boundary.

Unlike all of the other transformations we've seen so far, this transformation does not return a `DataFrame`.
* In Scala it returns `RelationalGroupedDataset`
* In Python it returns `GroupedData`

This is because the call `groupBy(..)` is only 1/2 of the transformation.

To see the other half, we need to take a look at it's return type, `RelationalGroupedDataset`.

### RelationalGroupedDataset

If we take a look at the API docs for `RelationalGroupedDataset`, we can see that it supports the following aggregations:

| Method | Description |
|--------|-------------|
| `avg(..)` | Compute the mean value for each numeric columns for each group. |
| `count(..)` | Count the number of rows for each group. |
| `sum(..)` | Compute the sum for each numeric columns for each group. |
| `min(..)` | Compute the min value for each numeric column for each group. |
| `max(..)` | Compute the max value for each numeric columns for each group. |
| `mean(..)` | Compute the average value for each numeric columns for each group. |
| `agg(..)` | Compute aggregates by specifying a series of aggregate columns. |
| `pivot(..)` | Pivots a column of the current DataFrame and perform the specified aggregation. |

With the exception of `pivot(..)`, each of these functions return our new `DataFrame`.

Together, `groupBy(..)` and `RelationalGroupedDataset` (or `GroupedData` in Python) give us what we need to answer some basic questions.

For Example, how many more requests did the desktop site receive than the mobile site receive?

For this all we need to do is group all records by **site** and then sum all the requests.

In [0]:
display(
  pageviewsDF
    .groupBy( col("site") )
    .sum()
)

site,sum(requests)
desktop,8737180972
mobile,4605797962


Notice above that we didn't actually specify which column we were summing....

In this case you will actually receive a total for all numerical values.

There is a performance catch to that - if I have 2, 5, 10? columns, then they will all be summed and I may only need one.

I can first reduce my columns to those that I wanted or I can simply specify which column(s) to sum up.

In [0]:
display(
  pageviewsDF
    .groupBy( col("site") )
    .sum("requests")
)

site,sum(requests)
desktop,8737180972
mobile,4605797962


And because I don't like the resulting column name, **sum(requests)** I can easily rename it...

In [0]:
display(
  pageviewsDF
    .groupBy( col("site") )
    .sum("requests")
    .withColumnRenamed("sum(requests)", "totalRequests")
)

site,totalRequests
desktop,8737180972
mobile,4605797962


How about the total number of requests per site? mobile vs desktop?

In [0]:
display(
  pageviewsDF
    .groupBy( col("site") )
    .count()
)

site,count
desktop,3600000
mobile,3600000


This result shouldn't surprise us... there were after all one record, per second, per site....

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) sum(), count(), avg(), min(), max()

The `groupBy(..)` operation is not our only option for aggregating.

The `...sql.functions` package actually defines a large number of aggregate functions
* `org.apache.spark.sql.functions` in the case of Scala & Java
* `pyspark.sql.functions` in the case of Python


Let's take a look at this in the Scala API docs (only because the documentation is a little easier to read).

Let's take a look at our last two examples... 

We saw the count of records and the sum of records.

Let's take do this a slightly different way...

This time with the `...sql.functions` operations.

And just for fun, let's throw in the average, minimum and maximum

In [0]:
(pageviewsDF
  .filter("site = 'mobile'")
  .select( sum( col("requests")), count(col("requests")), avg(col("requests")), min(col("requests")), max(col("requests")) )
  .show()
)
          
(pageviewsDF
  .filter("site = 'desktop'")
  .select( sum( col("requests")), count(col("requests")), avg(col("requests")), min(col("requests")), max(col("requests")) )
  .show()
)

+-------------+---------------+------------------+-------------+-------------+
|sum(requests)|count(requests)|     avg(requests)|min(requests)|max(requests)|
+-------------+---------------+------------------+-------------+-------------+
|   4605797962|        3600000|1279.3883227777778|          645|         3292|
+-------------+---------------+------------------+-------------+-------------+

+-------------+---------------+------------------+-------------+-------------+
|sum(requests)|count(requests)|     avg(requests)|min(requests)|max(requests)|
+-------------+---------------+------------------+-------------+-------------+
|   8737180972|        3600000|2426.9947144444445|         1312|         5695|
+-------------+---------------+------------------+-------------+-------------+



And let's just address one more pet-peeve...

Was that 3.6M records or 360K records?

In [0]:
(pageviewsDF
  .filter("site = 'mobile'")
  .select( 
    format_number(sum(col("requests")), 0).alias("sum"), 
    format_number(count(col("requests")), 0).alias("count"), 
    format_number(avg(col("requests")), 2).alias("avg"), 
    format_number(min(col("requests")), 0).alias("min"), 
    format_number(max(col("requests")), 0).alias("max") 
  )
  .show()
)

(pageviewsDF
  .filter("site = 'desktop'")
  .select( 
    format_number(sum(col("requests")), 0), 
    format_number(count(col("requests")), 0), 
    format_number(avg(col("requests")), 2), 
    format_number(min(col("requests")), 0), 
    format_number(max(col("requests")), 0) 
  )
  .show()
)

+-------------+---------+--------+---+-----+
|          sum|    count|     avg|min|  max|
+-------------+---------+--------+---+-----+
|4,605,797,962|3,600,000|1,279.39|645|3,292|
+-------------+---------+--------+---+-----+

+-------------------------------+---------------------------------+-------------------------------+-------------------------------+-------------------------------+
|format_number(sum(requests), 0)|format_number(count(requests), 0)|format_number(avg(requests), 2)|format_number(min(requests), 0)|format_number(max(requests), 0)|
+-------------------------------+---------------------------------+-------------------------------+-------------------------------+-------------------------------+
|                  8,737,180,972|                        3,600,000|                       2,426.99|                          1,312|                          5,695|
+-------------------------------+---------------------------------+-------------------------------+-------------------